In [1]:
import argparse
from collections import defaultdict
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import scanpy as sp
from scipy.stats import spearmanr
from sklearn.cluster import KMeans
import seaborn as sns
from tqdm import tqdm

In [2]:
# score = "ReadZS_norm"
# score2 = "ReadZS_ge_norm"

score = "SpliZ_norm"
score2 = "ge_norm"

name_dict = {"Myl6" : "Myl6", "Rps24" : "Rps24", "Gng13" : "Gng13", "chr11_10983_plus" : "Gpx3", "chr17_16275_minus" : "Slc8a1", "chr8_11013_plus" : "Gpm6a", "chr5_28581_minus" : "Actb", "chr4_23431_minus" : "Rps8", "chr1_30673_plus" : "S100a9"}

outpath = "output/rev_genes/"

out = {"gene" : [], "window" : [], "dataname" : [], "rev" : [], "corr" : [], "pval" : []}

scores = pd.read_csv("output/make_samplesheet/scores.csv",index_col=0)
srow = scores.loc[score]
srow2 = scores.loc[score2]
samples = pd.read_csv("output/make_samplesheet/spatial.csv",index_col = 0)

datanames = ["V1_Mouse_Kidney","V1_Mouse_Brain_Sagittal_Anterior","V1_Mouse_Brain_Sagittal_Anterior_Section_2",
            "V1_Mouse_Brain_Sagittal_Posterior","V1_Mouse_Brain_Sagittal_Posterior_Section_2", "p20218_s002_L2",
            "p20218_s001_L1"]

# datanames = ["V1_Mouse_Kidney"]
for dataname in datanames:

  
  # samples
  row = samples.loc[dataname]

  if row[srow["valname"]].endswith(".pq"):
    spliz_df = pd.read_parquet(row[srow["valname"]])
  else:
    spliz_df = pd.read_csv(row[srow["valname"]],sep="\t")



  if row[srow2["valname"]].endswith(".pq"):
    spliz_df2 = pd.read_parquet(row[srow2["valname"]])
  else:
    spliz_df2 = pd.read_csv(row[srow2["valname"]],sep="\t")

  spliz_df["gene_cell"] = spliz_df[srow["genecol"]] + "_" + spliz_df[srow["cellid"]]
  spliz_df2["gene_cell"] = spliz_df2[srow2["genecol"]] + "_" + spliz_df2[srow2["cellid"]]

  # get rid of outliers
  while (list(spliz_df["plot_xcoord"].sort_values().unique())[1] - list(spliz_df["plot_xcoord"].sort_values().unique())[0]) > 1000:

    spliz_df = spliz_df[spliz_df["plot_xcoord"] > list(spliz_df["plot_xcoord"].sort_values())[0]]

  while (list(spliz_df["plot_xcoord"].sort_values().unique())[-1] - list(spliz_df["plot_xcoord"].sort_values().unique())[-2]) > 1000:

    spliz_df = spliz_df[spliz_df["plot_xcoord"] < list(spliz_df["plot_xcoord"].sort_values())[-1]]

    
  # get rid of outliers
  while (list(spliz_df2["plot_xcoord"].sort_values().unique())[1] - list(spliz_df2["plot_xcoord"].sort_values().unique())[0]) > 1000:

    spliz_df2 = spliz_df2[spliz_df2["plot_xcoord"] > list(spliz_df2["plot_xcoord"].sort_values())[0]]

  while (list(spliz_df2["plot_xcoord"].sort_values().unique())[-1] - list(spliz_df2["plot_xcoord"].sort_values().unique())[-2]) > 1000:

    spliz_df2 = spliz_df2[spliz_df2["plot_xcoord"] < list(spliz_df2["plot_xcoord"].sort_values())[-1]]

  
  for gene in name_dict.keys():
    print(gene)
    gene_df = spliz_df[spliz_df[srow["genecol"]] == gene]
    print(gene_df.shape)
    gene_df2 = spliz_df2[spliz_df2[srow2["genecol"]] == gene]
    print(gene_df2.shape)
    df = gene_df[["barcode",srow["col"]]].merge(gene_df2[["barcode",srow2["col"]]],on="barcode").dropna()
    if df.shape[0] > 0:
      pval = spearmanr(df[[srow2["col"],srow["col"]]].dropna())
      if pval.correlation < 0:
        rev = True
      else:
        rev = False
      print("{} reverse: {} {}".format(dataname, rev, gene))
      out["gene"].append(name_dict[gene])
      out["window"].append(gene)
      out["dataname"].append(dataname)
      out["rev"].append(rev)
      out["corr"].append(pval.correlation)
      out["pval"].append(pval.pvalue)
    
out = pd.DataFrame.from_dict(out)
out.to_csv("{}rev_table_{}_{}.tsv".format(outpath,score, score2),sep="\t",index=False)

Myl6
(1426, 34)
(1426, 34)
V1_Mouse_Kidney reverse: False Myl6
Rps24
(1429, 34)
(1429, 34)
V1_Mouse_Kidney reverse: True Rps24
Gng13
(0, 34)
(0, 34)
chr11_10983_plus
(0, 34)
(0, 34)
chr17_16275_minus
(0, 34)
(0, 34)
chr8_11013_plus
(0, 34)
(0, 34)
chr5_28581_minus
(0, 34)
(0, 34)
chr4_23431_minus
(0, 34)
(0, 34)
chr1_30673_plus
(0, 34)
(0, 34)
Myl6
(2649, 34)
(2649, 34)
V1_Mouse_Brain_Sagittal_Anterior reverse: True Myl6
Rps24
(2690, 34)
(2690, 34)
V1_Mouse_Brain_Sagittal_Anterior reverse: False Rps24
Gng13
(1871, 34)
(1871, 34)
V1_Mouse_Brain_Sagittal_Anterior reverse: False Gng13
chr11_10983_plus
(0, 34)
(0, 34)
chr17_16275_minus
(0, 34)
(0, 34)
chr8_11013_plus
(0, 34)
(0, 34)
chr5_28581_minus
(0, 34)
(0, 34)
chr4_23431_minus
(0, 34)
(0, 34)
chr1_30673_plus
(0, 34)
(0, 34)
Myl6
(2793, 34)
(2793, 34)
V1_Mouse_Brain_Sagittal_Anterior_Section_2 reverse: True Myl6
Rps24
(2824, 34)
(2824, 34)
V1_Mouse_Brain_Sagittal_Anterior_Section_2 reverse: False Rps24
Gng13
(2240, 34)
(2240, 34)
V1_Mo

In [3]:
out

,gene,window,dataname,rev,corr,pval
0,Myl6,Myl6,V1_Mouse_Kidney,False,0.219166,5.715037e-17
1,Rps24,Rps24,V1_Mouse_Kidney,True,-0.261107,1.054739e-23
2,Myl6,Myl6,V1_Mouse_Brain_Sagittal_Anterior,True,-0.007159,7.126371e-01
3,Rps24,Rps24,V1_Mouse_Brain_Sagittal_Anterior,False,0.104825,5.057752e-08
4,Gng13,Gng13,V1_Mouse_Brain_Sagittal_Anterior,False,0.007159,7.569678e-01
5,Myl6,Myl6,V1_Mouse_Brain_Sagittal_Anterior_Section_2,True,-0.017454,3.564767e-01
6,Rps24,Rps24,V1_Mouse_Brain_Sagittal_Anterior_Section_2,False,0.076015,5.263710e-05
7,Gng13,Gng13,V1_Mouse_Brain_Sagittal_Anterior_Section_2,False,0.214001,1.290829e-24
8,Myl6,Myl6,V1_Mouse_Brain_Sagittal_Posterior,False,0.041327,1.945065e-02
9,Rps24,Rps24,V1_Mouse_Brain_Sagittal_Posterior,False,0.127660,1.457778e-13
